# Variational Quantum Circuit

Variational Quantum Circuit or Parametrized Quantum Circuits are an important ingredient in Variational Algorithms like Variational Quantum Eigensolvers. They act as a bridge between the quantum and classical part of these hybrid variational algorithms. 

We generally, start with all zero states $|0\rangle^{\otimes n}$ and apply a quantum circuit with rotation gates $U_\theta$ with free parameters so that we can later tune them. 

$$|\psi_\theta\rangle=U_\theta|\phi \rangle$$

We obtain the cost function from the expectation value of these circuit. This cost function is classically optimized by tuning the parameters $\theta = (\theta_1 , \theta_2 \dots) $.

# ADD AN ILLUSTRATION

These circuits are very crucial in the NISQ era because they contain low number of gates. But we have to remember that for these algorithms to work efficiently we have to choose an effective circuit that well represents the solution space while maintaining a low circuit depth and number of parameters.

## Example to understand this in detail

### Implement a variational circuit that returns $|0\rangle$ and $|1\rangle$ with equal probability.

#### Importing the libraries (Using Pennylane for this example)

In [ ]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt

In this problem we have to essentially create the state $ \tfrac{1}{\sqrt{2}}(|0\rangle + e^{i\theta}|1\rangle)$, where $e^{i\theta}$ is the global phase from $|0\rangle$. We already know that the conversion from $|0\rangle$ state to $\tfrac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$ can be easily achieved by Hadamard gate. But for the sake of understanding this problem we will use rotation gates.

#### Creating the variational circuit

In [ ]:
number_qubits = 1

dev = qml.device("qiskit.aer", wires=number_qubits, shots=1000, analytic=False)

@qml.qnode(dev)
def state(theta):
    
    qml.RY(theta[0], wires=0)
    
    return qml.probs(wires=[0])


### ADD A PHOTO OF CIRCUIT

#### Creating the cost function

`qml.probs()` returns a flat array or tensor containing the (marginal) probabilities of each quantum state in the lexigrophic ordering. 

Hence the first value of the array corresponds to probability of the state '0' and the second value to the state '1'.

Both the states should be equally probable, therefore, the probability of the following states should be:
$$|0\rangle \rightarrow 0.5 $$
$$|1\rangle \rightarrow 0.5 $$

The cost function should be,
 `((prob_0-0.5)**2 + (prob_1-0.5)**2) `.
 Minimizing this will get our results.

In [ ]:
def cost(theta):           #cost function for noisy simulation
    prob_0, prob_1 = state(theta)
    return ((prob_0-0.5)**2 + (prob_1-0.5)**2)

#### Finding the optimal paremeters using classical optimizer (Gradient Descent)

In [ ]:
for num_shots in [1, 10, 100, 1000]:
    dev.shots = num_shots
    initial_parameters = np.random.randn(1) #starting with random parameters
    
    steps = 40 # Number of steps of optimization
    
    parameters = initial_parameters.copy()
    
    optimizer = qml.GradientDescentOptimizer(stepsize=0.6) #The Gradient Descent Optimizer
    
    costs = []
    
    print('--------------------------------\n')
    print('For {0} shots\n'.format(num_shots))
    for i in range(steps):
        costs.append(cost(parameters))
        parameters = optimizer.step(cost, parameters)
        
        if (i%10 == 0):
            print('The cost for {0} measurement sampling is {1}\n'.format(num_shots, cost(parameters)))
            print('The parameters are {0}\n'.format(parameters))
    all_costs.append(costs)

    p = [0, 0]
    p[0], p[1] = state(parameters)
    print('Final probability of states is as follows: \n')
    print('\n Probality of 0 is: {0} \n Probality of 1 is: {1}'.format(p[0], p[1]))
    prob.append(p)

In [ ]:
print('We get the final parameter as {0}.'.format(parameters))

### Probability Distribution

In [ ]:
plt.bar(['0', '1'], prob[0]) #Probability Distribution for shots 1

In [ ]:
plt.bar(['0', '1'], prob[1]) #Probability Distribution for shots 10

In [ ]:
plt.bar(['0', '1'], prob[2]) #Probability Distribution for shots 100

In [ ]:
plt.bar(['0', '1'], prob[3]) #Probability Distribution for shots 1000